In [1]:
import grid2op
import torch
import warnings
from dreamer.modules.encoder import Encoder
from dreamer.modules.decoder import Decoder
from dreamer.modules.recurrentModel import RecurrentModel
from dreamer.modules.rewardModel import RewardPredictor
warnings.filterwarnings('ignore')

In [2]:
env = grid2op.make("l2rpn_case14_sandbox")

In [3]:
env.set_id(0)
obs = env.reset()
env.fast_forward_chronics(1094)

In [5]:
env.chronics_handler.

(datetime.datetime(2019, 1, 9, 19, 15),
 {'injection': {'load_p': array([ 23.1, 110.5,  53.2,   8.7,  12.2,  34. ,  10.8,   4.2,   6.8,
           16.1,  17.6], dtype=float32),
   'load_q': array([16.4, 78.9, 37.4,  6. ,  8.7, 23.5,  7.6,  2.9,  5. , 11.3, 12.4],
         dtype=float32),
   'prod_p': array([100.2,  95.7,  11.2,   0. ,   0. ,  98.1], dtype=float32)}},
 array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1]),
 array([142.1, 142.1,  22. ,  22. ,  13.2, 142.1], dtype=float32))

In [3]:
obs = env.reset()
obs.size()

467

In [6]:
print(obs.to_vect())

[ 2.0190000e+03  1.0000000e+00  6.0000000e+00  0.0000000e+00
  0.0000000e+00  6.0000000e+00  8.1400002e+01  7.9300003e+01
  5.3000002e+00  0.0000000e+00  0.0000000e+00  8.2246674e+01
  1.9496038e+01  7.1340233e+01  2.4368923e+01  2.4368923e+01
  2.4018070e+01 -1.7274660e+01  1.4210001e+02  1.4210001e+02
  2.2000000e+01  2.2000000e+01  1.3200001e+01  1.4210001e+02
  2.1900000e+01  8.5800003e+01  4.4299999e+01  6.9000001e+00
  1.1900000e+01  2.8500000e+01  8.8000002e+00  3.5000000e+00
  5.4000001e+00  1.2600000e+01  1.4400000e+01  1.5400000e+01
  5.9700001e+01  3.0799999e+01  4.8000002e+00  8.3000002e+00
  1.9400000e+01  6.0999999e+00  2.4000001e+00  3.9000001e+00
  8.8000002e+00  1.0500000e+01  1.4210001e+02  1.4210001e+02
  1.3866075e+02  1.3929695e+02  2.2000000e+01  2.1130220e+01
  2.1129551e+01  2.1478817e+01  2.1571596e+01  2.1432823e+01
  2.0750198e+01  4.2346096e+01  3.9900578e+01  2.3991766e+01
  4.1828262e+01  3.5666172e+01  1.7225140e+01 -2.7542929e+01
  8.1183472e+00  7.46021

In [8]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

config = AttrDict(
    latent_dim = 64,
    action_dim = 176,
    hidden_dim = 64,
    output = 64,
    input_dim = obs.size()
)

In [9]:
config.input_dim

467

In [11]:
e = Encoder(config)
d = Decoder(config)
rec = RecurrentModel(config)

In [15]:
obs = torch.tensor(obs.to_vect())
obs = obs.unsqueeze(0)
latent = e(obs)

In [18]:
ori = d(latent)
ori.shape

torch.Size([1, 467])

In [22]:
h = torch.tensor([0]).unsqueeze(0)
a = torch.tensor([0]).unsqueeze(0)


tensor([[0]])

In [23]:
z_0 = torch.randn(config.latent_dim)           # Initial latent state from encoder (randomly initialized here)
a_0 = torch.zeros(config.action_dim)           # Initial action is set to zero
h_0 = torch.zeros(config.hidden_dim)

In [31]:
z = latent.squeeze(0)
z.shape

torch.Size([64])

In [32]:
h_1 = rec(z, a_0, h_0)
print("Shape of h_1:", h_1.shape)  # Should be (hidden_dim,)


Shape of h_1: torch.Size([64])


In [29]:
a_0.shape

torch.Size([176])

In [7]:
import os
import numpy as np
import grid2op
import random
import pandas as pd
import time

In [29]:
def topology_greedy_action(env, substation_ids):
    """
    Attempts to stabilize the grid by applying topology actions on specified substations.
    
    Parameters:
    - env: Grid2Op environment instance
    - substation_ids: List of IDs of substations to apply topology actions on

    Returns:
    - action_chosen: The best topology action found to stabilize the grid, or a "do nothing" action if none is found
    """
    actions = []
    obs = env.get_obs()
    min_rho = obs.rho.max()
    action_chosen = env.action_space({})  # start with a "do nothing" action

    actions_1 = env.action_space.get_all_unitary_topologies_change(env.action_space, sub_id=substation_ids[0])
    actions_2 = env.action_space.get_all_unitary_topologies_change(env.action_space, sub_id=substation_ids[1])
    all_actions = actions_1 + actions_2

    
    # Start time tracking for performance measurement
    start_time = time.time()

    # Loop through all possible unitary topology changes
    for action in all_actions:
        
        # Check if the action is legal
        if not env._game_rules(action, env):
            continue

        # Simulate the action
        obs_, _, done, _ = obs.simulate(action)
        actions.append(action)
        
        # If the grid is not in a failed state and the action reduces congestion
        if (not done) and (obs_.rho.max() < min_rho):
            min_rho = obs_.rho.max()
            action_chosen = action  # Update the best action

    # Logging the best action found
    print("Greedy topology action found with max rho reduced to %.5f, search duration: %.2f seconds" %
          (min_rho, time.time() - start_time))
    
    return action_chosen, len(actions)

In [33]:
def attack_and_stabilize(env, line_to_attack):
    """
    Disconnects a specified line and attempts to stabilize the grid using topology actions on connected substations.

    Parameters:
    - env: Grid2Op environment instance
    - line_to_attack: The ID of the line to be disconnected as part of the attack
    """
    # Get the origin and extremity substations of the line to attack
    origin_substation = env.line_or_to_subid[line_to_attack]
    extremity_substation = env.line_ex_to_subid[line_to_attack]

    # Log the attacked line and connected substations
    print(f"Attacking line-{line_to_attack}, connected between substations {origin_substation} and {extremity_substation}")

    # Initial grid observation
    obs = env.get_obs()

    # Disconnect the specified line
    line_status = np.zeros(obs.rho.shape, dtype=np.int32)
    line_status[line_to_attack] = -1  # -1 indicates disconnection
    action_disconnect = env.action_space({"set_line_status": line_status})
    obs, _, done, _ = env.step(action_disconnect)
    
    # Check if the grid is stable after disconnecting the line
    if done:
        print(f"Grid failed after disconnecting line {line_to_attack}. Unable to stabilize.")
        return

    # Apply the greedy topology action to stabilize the grid, if possible
    action_stabilize, l_act = topology_greedy_action(env, [origin_substation, extremity_substation])
    obs, _, done, _ = env.step(action_stabilize)

    # Check final grid state after attempting to stabilize
    if done:
        print("Grid failed after applying topology action.")
        print(l_act)
        print("\n ############################################################## \n", action_stabilize)
    else:
        print("Topology action successfully applied to reduce overload.")
        print(l_act)
        print("\n ############################################################## \n", action_stabilize)


In [34]:
env = grid2op.make("l2rpn_wcci_2022")
env.reset()
action = attack_and_stabilize(env, line_to_attack=9)

Attacking line-9, connected between substations 68 and 74
Greedy topology action found with max rho reduced to 0.84682, search duration: 11.21 seconds
Topology action successfully applied to reduce overload.
94

 ############################################################## 
 This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - Change the bus of the following element(s):
	 	 - Switch bus of line (extremity) id 184 [on substation 68]
	 - NOT force any particular bus configuration


In [32]:
print(action)

None
